In [1]:
import gc
import sys
sys.path.append('../')
import xlrd
import math
import csv
import json
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
# import seaborn as sns

from pathlib import Path
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tqdm import tqdm
# from collections import defaultdict

%matplotlib inline



In [2]:
path_dataset_all = 'AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient_all_data_set/'
path_dataset_oCare = 'AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/Patient_oCare_data_file/'


In [5]:
with open(path_dataset_all+'bp_data_1024.csv', encoding='utf-8') as f:
    inp = f.readlines()

bp_data_1020 = pd.DataFrame([row.strip().split(',') for row in inp[1:]], columns=inp[0].strip().split(','))

for c in bp_data_1020.columns:
    bp_data_1020[c] = bp_data_1020[c].apply(lambda x: x.encode('utf-8').decode('utf-8-sig'))  # https://www.itread01.com/content/1518084126.html

bp_data_1020.columns = [c.encode('utf-8').decode('utf-8-sig') for c in bp_data_1020.columns]
bp_data_1020['diastolic'] = np.where(bp_data_1020['diastolic']=='', '68', bp_data_1020['diastolic'])

for c in bp_data_1020.columns:
    if c not in ['hmpcrtno','btrim','watch']:
#         bp_data_1020[bp_data_1020[c]=='']
        bp_data_1020[c] = bp_data_1020[c].astype(int)

bp_data_1020.shape
bp_data_1020.head()
bp_data_1020.isnull().sum()


machine_data_1018 = pd.read_csv(path_dataset_all+'machine_data_1024.csv')

for c in machine_data_1018.columns:
    if c not in ['hmpcrtno','btrim','ufprofile','sodiumpfile','watcg']:
        machine_data_1018[c] = machine_data_1018[c].astype(float)
        machine_data_1018[c] = machine_data_1018[c].interpolate(method='linear')
    else:
        machine_data_1018[c] = machine_data_1018[c].fillna(0.0)

machine_data_1018.shape
machine_data_1018.head()
machine_data_1018.isnull().sum()


###合併所有oCare data到一個dataframe，先以 3376 為例
hmpcrt_no = [3376,7146,10380,15521,16827,18651,23278,31384,32813,44488,45934,
             56075,57830,63579,90496,96961,119298,139980,142533,184883,216995,221211,221472,258260]

default_columns = oCare_data.columns
oCare_data = pd.DataFrame([[0]*len(default_columns)], columns=default_columns)

for no in tqdm(hmpcrt_no):
    hmpcrt_folder = Path(path_dataset_oCare + str(no))
    all_hmpcrt_data_file = list(hmpcrt_folder.glob("**/*.csv"))
    
    for hmpcrt_data_file in all_hmpcrt_data_file:

        with open(hmpcrt_data_file) as f:
            inp = f.readlines()

#         oCare_data = pd.DataFrame([row for row in [row.replace('\n', '').replace('"','').split(',') for row in inp[1:]] if row != ['']], 
#                            columns=inp[0].replace('\n', '').replace('"','').split(','))
        oCare_data = pd.concat([oCare_data, pd.DataFrame([row for row in [row.replace('\n', '').replace('"','').split(',') for row in inp[1:]] if row != ['']], 
                           columns=inp[0].replace('\n', '').replace('"','').split(','))])

oCare_data = oCare_data.iloc[1:oCare_data.shape[0], :]
for c in oCare_data.columns:
    oCare_data[c] = oCare_data[c].astype(float)
    oCare_data[c] = oCare_data[c].astype(int)
oCare_data.shape
oCare_data.head()
oCare_data.isnull().sum()


(5066, 11)

,hmpcrtno,btrim,daidate,daitime,systole,map,diastolic,pulserate,watch,Hypo_delta_20,Hypo_90mmHg
0,3376,B05,1090821,73759,209,125,66,82,5,0,0
1,3376,B05,1090821,82758,171,89,55,74,5,1,0
2,3376,B05,1090821,92559,187,109,65,75,5,0,0
3,3376,B05,1090821,102959,174,96,59,76,5,0,0
4,3376,B05,1090821,105759,131,76,49,76,5,1,0


hmpcrtno         0
btrim            0
daidate          0
daitime          0
systole          0
map              0
diastolic        0
pulserate        0
watch            0
Hypo_delta_20    0
Hypo_90mmHg      0
dtype: int64

(81241, 31)

,hmpcrtno,start_time,end_time,btrim,daidate,daitime,vpressure,apressure,tmp,abf,ttemper,atemper,conductivity,ufrate,uftarget,ufvolume,uftime,flowrate,currflowrate,bloodleakv,cloudingv,cyclicpht,targetsodium,bicarbonatea,effectivebf,accumulatbv,startsodium,ufprofile,sodiumpfile,hpdelrate,watch
0,3376,736.0,1106.0,B05,1090821.0,74159.0,198.0,-2.0,49.0,249.0,35.5,35.6,13.7,0.4,1.3,0.0,205.0,500.0,521.0,51.0,49.0,0.0,1380.0,0.0,248.0,10.0,0.0,0.0,0.0,10.0,5.0
1,3376,736.0,1106.0,B05,1090821.0,74359.0,204.0,0.0,46.0,249.0,35.5,35.6,13.7,0.4,1.3,0.0,203.0,500.0,521.0,50.0,49.0,0.0,1380.0,0.0,249.0,15.0,0.0,0.0,0.0,10.0,5.0
2,3376,736.0,1106.0,B05,1090821.0,74559.0,195.0,0.0,50.0,249.0,35.5,35.6,13.7,0.4,1.3,0.0,201.0,500.0,521.0,51.0,49.0,0.0,1380.0,0.0,249.0,20.0,0.0,0.0,0.0,10.0,5.0
3,3376,736.0,1106.0,B05,1090821.0,74759.0,189.0,0.0,52.0,249.0,35.5,35.6,13.7,0.4,1.3,0.1,199.0,500.0,521.0,51.0,49.0,0.0,1380.0,0.0,248.0,25.0,0.0,0.0,0.0,10.0,5.0
4,3376,736.0,1106.0,B05,1090821.0,74959.0,180.0,0.0,52.0,249.0,35.5,35.6,13.7,0.4,1.3,0.1,196.0,500.0,521.0,51.0,49.0,750.0,1380.0,0.0,248.0,31.0,0.0,0.0,0.0,10.0,5.0


hmpcrtno        0
start_time      0
end_time        0
btrim           0
daidate         0
daitime         0
vpressure       0
apressure       0
tmp             0
abf             0
ttemper         0
atemper         0
conductivity    0
ufrate          0
uftarget        0
ufvolume        0
uftime          0
flowrate        0
currflowrate    0
bloodleakv      0
cloudingv       0
cyclicpht       0
targetsodium    0
bicarbonatea    0
effectivebf     0
accumulatbv     0
startsodium     0
ufprofile       0
sodiumpfile     0
hpdelrate       0
watch           0
dtype: int64

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [01:23<00:00,  3.49s/it]


(3741865, 5)

,hmpcrtno,daidate,daitime,SpO2,HR
0,3376,1090824,75757,86,88
1,3376,1090824,75758,87,87
2,3376,1090824,75759,87,87
3,3376,1090824,75800,87,87
4,3376,1090824,75801,88,86


hmpcrtno    0
daidate     0
daitime     0
SpO2        0
HR          0
dtype: int64

In [7]:
###machine_data aggregate 成 10 分鐘一筆

for hmpcrt_no in tqdm(machine_data_1018.hmpcrtno.unique()[1:]):
    if hmpcrt_no not in [3376, 31384]:
#         tmp_bp = bp_data_1020[bp_data_1020['hmpcrtno']==str(hmpcrt_no)]
        tmp_machine = machine_data_1018[machine_data_1018['hmpcrtno']==hmpcrt_no]
        print(tmp_machine.shape)

        tmp_machine['day_stamp'] = tmp_machine['daidate'].apply(lambda x: (dt.datetime.strptime('0'+str(int(x)), '%Y%m%d')-dt.datetime.strptime('0'+str(int(tmp_machine['daidate'].values[0])), '%Y%m%d')).days)

        list_timeOfDay = list()
        for daidate in tqdm(tmp_machine.daidate.unique()):
            tmp = tmp_machine[tmp_machine['daidate']==daidate].daitime.values
            tmp = [round((dt.datetime.strptime(str(int(x)), '%H%M%S') - dt.datetime.strptime(str(int(tmp[0])), '%H%M%S')).seconds / 60, 0) for x in tmp]
            for item in tmp:
                list_timeOfDay.append(item)

        tmp_machine['time_of_day'] = list_timeOfDay
        duration_base = 30
        tmp_machine['time_stamp'] = tmp_machine['time_of_day'].apply(lambda x: math.floor(x / duration_base))
        timestamptable = tmp_machine[['day_stamp','time_stamp','daidate','daitime']]
        timestamptable = timestamptable.groupby(['day_stamp','time_stamp']).first().reset_index()

        tmp_machine = tmp_machine.drop(['start_time','end_time','btrim','daidate','daitime','time_of_day'], axis=1)
        tmp_machine = tmp_machine.groupby(['hmpcrtno','day_stamp','time_stamp']).mean().reset_index()
    #     machine_data_aggre_10min = pd.merge(tmp_machine, timestamptable, on=['day_stamp','time_stamp'], how='left')
        machine_data_aggre_10min = pd.concat([tmp_machine, pd.merge(tmp_machine, timestamptable, on=['day_stamp','time_stamp'], how='left')])

machine_data_aggre_10min.shape
machine_data_aggre_10min.head()
machine_data_aggre_10min.isnull().sum()
machine_data_aggre_10min.to_csv(f'machine_data_1024_aggregate_by{str(duration_base)}min.csv', index=False)


  0%|                                                                                                                                                                                                                | 0/23 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 380.53it/s]
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarni

(2698, 31)
(3256, 31)


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 351.91it/s]
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

(2541, 31)
(5388, 31)


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 223.43it/s]
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

...

(2201, 31)
(3235, 31)


C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 352.00it/s]
C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

(430, 30)

,hmpcrtno,day_stamp,time_stamp,vpressure,apressure,tmp,abf,ttemper,atemper,conductivity,ufrate,uftarget,ufvolume,uftime,flowrate,currflowrate,bloodleakv,cloudingv,cyclicpht,targetsodium,bicarbonatea,effectivebf,accumulatbv,startsodium,ufprofile,sodiumpfile,hpdelrate,watch,daidate,daitime
0,258260,0,0,159.846154,-0.615385,52.307692,245.153846,36.0,36.100000,14.153846,0.4,1.5,0.076923,227.000000,500.0,527.769231,50.846154,48.692308,346.153846,1398.461538,1.0,240.076923,30.230769,0.0,0.0,0.0,9.230769,3.0,NaN,NaN
1,258260,0,1,161.142857,-0.285714,52.928571,249.000000,36.0,36.085714,13.800000,0.4,1.5,0.250000,197.642857,500.0,521.642857,51.000000,48.285714,696.428571,1390.000000,1.0,248.571429,103.285714,0.0,0.0,0.0,10.000000,3.0,NaN,NaN
2,258260,0,2,188.285714,-0.142857,51.428571,249.000000,36.0,36.092857,13.800000,0.4,1.5,0.450000,167.785714,500.0,521.071429,51.000000,48.357143,695.500000,1390.000000,1.0,248.428571,177.500000,0.0,0.0,0.0,10.000000,3.0,NaN,NaN
3,258260,0,3,222.428571,-0.428571,52.571429,248.785714,36.0,36.050000,13.800000,0.4,1.5,0.635714,138.000000,500.0,518.785714,51.000000,48.714286,695.642857,1390.000000,1.0,248.714286,251.785714,0.0,0.0,0.0,10.000000,3.0,NaN,NaN
4,258260,0,4,206.571429,-0.142857,50.785714,249.000000,36.0,36.050000,13.800000,0.4,1.5,0.821429,108.000000,500.0,512.285714,51.000000,48.785714,750.000000,1390.000000,1.0,248.857143,326.500000,0.0,0.0,0.0,10.000000,3.0,NaN,NaN


hmpcrtno          0
day_stamp         0
time_stamp        0
vpressure         0
apressure         0
tmp               0
abf               0
ttemper           0
atemper           0
conductivity      0
ufrate            0
uftarget          0
ufvolume          0
uftime            0
flowrate          0
currflowrate      0
bloodleakv        0
cloudingv         0
cyclicpht         0
targetsodium      0
bicarbonatea      0
effectivebf       0
accumulatbv       0
startsodium       0
ufprofile         0
sodiumpfile       0
hpdelrate         0
watch             0
daidate         215
daitime         215
dtype: int64

In [8]:
machine_data_aggre_10min = pd.read_csv(path_dataset_all+'machine_data_1024_aggregate_by30min.csv')
machine_data_aggre_10min.shape
machine_data_aggre_10min.head()


FileNotFoundError: [Errno 2] No such file or directory: 'AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient_all_data_set/machine_data_1024_aggregate_by30min.csv'

In [104]:
loop_daidate = 1090824
tmp_machine_match_daidate = tmp_machine[tmp_machine['daidate']==loop_daidate]
machine_daitime_match = tmp_machine_match_daidate.daitime.values

tmp_bp_match_daidate = tmp_bp[tmp_bp['daidate']==loop_daidate]
daitime_match = tmp_bp_match_daidate.daitime.unique().tolist()

tmp_oCare_match_daidate = oCare_data[oCare_data['daidate']==loop_daidate]


collect = dict()
collect[str(3376)] = list()
for point in range(len(machine_daitime_match)):
    if (machine_daitime_match[point] > min(daitime_match)) & (machine_daitime_match[point] < max(daitime_match)):
        index = next(x[0] for x in enumerate(daitime_match) if x[1] > machine_daitime_match[point]) - 1
    elif machine_daitime_match[point] <= min(daitime_match):
        index = 0
    elif machine_daitime_match[point] >= max(daitime_match):
        index = len(daitime_match) - 1
    else:
        print(hmpcrt, daidate, machine_daitime_match[point], daitime_match)
    
    if point == 0:
        period = tmp_oCare_match_daidate[tmp_oCare_match_daidate['daitime'] < machine_daitime_match[point]]
    else:
        period = tmp_oCare_match_daidate[(tmp_oCare_match_daidate['daitime'] > machine_daitime_match[point - 1] - 1) & (tmp_oCare_match_daidate['daitime'] < machine_daitime_match[point])]
    
    HR_mean = np.mean(period.HR)
    HR_cv = (np.std(period.HR) / HR_mean)

    SpO2_mean = np.mean(period.SpO2)
    SpO2_cv = (np.std(period.SpO2) / SpO2_mean)
    
    #machine_data
    to_append = tmp_machine_match_daidate[tmp_machine_match_daidate['daitime']==machine_daitime_match[point]].iloc[0, :].values.tolist()
    #bp_data
    for v in tmp_bp_match_daidate.iloc[index, 4:].values:
        to_append.append(v)
    #oCare_data
    for v in [HR_mean, HR_cv, SpO2_mean, SpO2_cv]:
        to_append.append(v)

    collect[str(3376)].append(to_append)
    
    
    
    
collect


{'3376': [[3.0,
   0.0,
   114.0,
   0.0,
   52.0,
   232.33333333333334,
   36.0,
   35.766666666666666,
   14.133333333333333,
   0.4000000000000001,
   1.5,
   0.0,
   207.33333333333334,
   500.0,
   531.6666666666666,
   51.0,
   49.0,
   0.0,
   1400.0,
   0.0,
   231.66666666666666,
   5.666666666666667,
   0.0,
   0.0,
   0.0,
   6.666666666666667,
   5.0,
   1090824.0,
   73423.0,
   169,
   116,
   59,
   86,
   '5',
   0,
   0,
   0,
   nan,
   nan,
   nan,
   nan],
  [3.0,
   1.0,
   174.6,
   -0.4,
   52.6,
   249.0,
   36.0,
   36.08,
   14.2,
   0.4,
   1.5,
   0.08,
   199.0,
   500.0,
   529.0,
   50.8,
   48.8,
   300.0,
   1400.0,
   0.0,
   248.6,
   26.0,
   0.0,
   0.0,
   0.0,
   10.0,
   5.0,
   1090824.0,
   74423.0,
   169,
   116,
   59,
   86,
   '5',
   0,
   0,
   0,
   nan,
   nan,
   nan,
   nan],
  [3.0,
   2.0,
   166.8,
   0.0,
   48.0,
   249.0,
   35.7,
   35.92,
   14.02,
   0.4,
   1.5,
   0.16,
   188.2,
   500.0,
   525.8,
   51.0,
   48.0,
   7

In [49]:
for hmpcrt in collect.keys():
    final_3376 = pd.DataFrame(collect[str(hmpcrt)], columns=tmp_machine_match_daidate.columns.tolist()[1:]+tmp_bp_match_daidate.columns.tolist()[4:]+['oCare_HR_mean', 'oCare_HR_cv', 'oCare_SpO2_mean', 'oCare_SpO2_cv'])
    final_3376.isnull().sum()


,hmpcrtno,daidate,daitime,SpO2,HR
0,0,0,0,0,0


In [82]:
oCare_data[(oCare_data['hmpcrtno']==3376)&(oCare_data['daidate']==1090824)&(oCare_data['daitime']>80400)][:30]

,hmpcrtno,daidate,daitime,SpO2,HR
165,3376,1090824,80401,99,84
166,3376,1090824,80402,99,84
167,3376,1090824,80403,99,84
168,3376,1090824,80404,99,85
169,3376,1090824,80405,99,85
170,3376,1090824,80406,99,85
171,3376,1090824,80407,99,85
172,3376,1090824,80408,99,85
173,3376,1090824,80409,99,85
174,3376,1090824,80410,99,85


### format-IV revise (v3~)
- filter NaN
- 補 oCare 缺值：有初始值用初始值補，無則用該位病人的平均值補


In [11]:
# result = pd.read_csv('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient-aggregate-features--format-IV.4.csv')
# # result.isnull().sum()
# result = result.drop(['watch.1'], axis=1)
# # result.head()

# # result_valid = result[result.notnull().all(1)]
# result_valid = result[result['previous_systole'].notnull()]
# result_valid = result_valid[result_valid['systole'].notnull()]

# ###補oCare缺值
# ####有初始值用初始值補，無則用該位病人的平均值補
# for hmpcrtno in tqdm(result_valid.hmpcrtno.unique()):
#     daystamplist = result_valid[result_valid['hmpcrtno']==hmpcrtno].daidate.unique()
#     for daystamp in daystamplist:
#         tmp = result_valid[(result_valid['hmpcrtno']==hmpcrtno)&(result_valid['daidate']==daystamp)]
#         for col in ['oCare_HR_mean','oCare_HR_cv','oCare_SpO2_mean','oCare_SpO2_cv']:
#             try:
#                 result_valid[col] = np.where((result_valid['hmpcrtno']==hmpcrtno)&(result_valid['daidate']==daystamp)&(result_valid[col].isnull()), tmp[tmp[col].notnull()][col].values[0], result_valid[col])
#             except:
#                 print(hmpcrtno, daystamp, col)
#                 pass

# for hmpcrtno in tqdm(result_valid.hmpcrtno.unique()):
#     for col in ['oCare_HR_mean','oCare_HR_cv','oCare_SpO2_mean','oCare_SpO2_cv']:
#         tmp_notnull = result_valid[(result_valid['hmpcrtno']==hmpcrtno)&(result_valid[col].notnull())]
#         result_valid[col] = np.where((result_valid['hmpcrtno']==hmpcrtno)&(result_valid[col].isnull()), np.mean(tmp_notnull[col].values), result_valid[col])
#         result_valid[col] = np.where((result_valid['hmpcrtno']==hmpcrtno)&(result_valid[col]==0), np.mean(tmp_notnull[col].values), result_valid[col])
        
# result_valid[(result_valid['hmpcrtno']==3376.0)].daidate.unique()
result_valid[(result_valid['hmpcrtno']==3376.0)&(result_valid['daidate']==1090826)].iloc[:, -4:][-30:]
###下次回來：其他缺值看能否用前或後順補 (df.interplote)
# result_valid.isnull().sum()
# result_valid.shape
# result_valid.head()


,oCare_HR_cv,oCare_SpO2_mean,oCare_SpO2_cv,match_index
257,0.028028,98.666667,0.004778,5
258,0.031783,99.000000,0.011861,5
259,0.034034,98.808333,0.003984,5
260,0.061774,98.000000,0.011861,5
261,0.027033,98.562500,0.006328,5
262,0.043324,97.650000,0.005230,5
263,0.018706,98.906780,0.005885,5
264,0.025027,98.641667,0.005521,5
265,0.134393,99.000000,0.011861,5
266,0.118806,99.000000,0.011861,5


In [59]:
def add_delta(result_valid, assign_column):
    
    delta_systole = list()
    
    for hmpcrtno in tqdm(result_valid.hmpcrtno.unique()):
        result_hmpcrtno = result_valid[(result_valid['hmpcrtno']==hmpcrtno)]

        for day_stamp in result_hmpcrtno.day_stamp.unique():
            tmp_list = result_hmpcrtno[result_hmpcrtno['day_stamp']==day_stamp][['hmpcrtno','day_stamp','time_stamp',str(assign_column)]][str(assign_column)].values
            tmp_list = [0.0,0.0] + (tmp_list[1:] - tmp_list[:-1]).tolist()[:-1]
            for item in tmp_list:
                delta_systole.append(item)

    result_valid[f'delta_{assign_column}'] = delta_systole
    return result_valid


bp_columns = ['systole','map','diastolic','pulserate','previous_systole']
machine_columns = ['vpressure','apressure','tmp','abf','ttemper','atemper','conductivity','ufrate','uftarget','ufvolume',
                   'uftime','flowrate','currflowrate','bloodleakv','cloudingv','cyclicpht','targetsodium','bicarbonatea',
                   'effectivebf','accumulatbv','startsodium','ufprofile','sodiumpfile','hpdelrate']
oCare_columns = ['oCare_HR_mean','oCare_HR_mean','oCare_HR_cv','oCare_SpO2_mean','oCare_SpO2_cv']

for c in tqdm(bp_columns):
    result_valid = add_delta(result_valid, assign_column=c)
for c in tqdm(machine_columns):
    result_valid = add_delta(result_valid, assign_column=c)
for c in tqdm(oCare_columns):
    result_valid = add_delta(result_valid, assign_column=c)

result_valid.head()


 25%|██████████████████████████████████████████████████                                                                                                                                                      | 6/24 [00:02<00:08,  2.02it/s]


 22%|███████████████████████████████████████████▍                                                                                                                                                            | 5/23 [00:00<00:00, 44.51it/s]


 43%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                | 10/23 [00:00<00:00, 44.46it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.01it/s]


,hmpcrtno,day_stamp,time_stamp,vpressure,apressure,tmp,abf,ttemper,atemper,conductivity,ufrate,uftarget,ufvolume,uftime,flowrate,currflowrate,bloodleakv,cloudingv,cyclicpht,targetsodium,bicarbonatea,effectivebf,accumulatbv,startsodium,ufprofile,sodiumpfile,hpdelrate,watch,daidate,daitime,systole,map,diastolic,pulserate,Hypo_delta_20,Hypo_90mmHg,previous_systole,oCare_HR_mean,oCare_HR_cv,oCare_SpO2_mean,oCare_SpO2_cv,delta_systole,delta_map,delta_diastolic,delta_pulserate,delta_previous_systole,delta_vpressure,delta_apressure,delta_tmp,delta_abf,delta_ttemper,delta_atemper,delta_conductivity,delta_ufrate,delta_uftarget,delta_ufvolume,delta_uftime,delta_flowrate,delta_currflowrate,delta_bloodleakv,delta_cloudingv,delta_cyclicpht,delta_targetsodium,delta_bicarbonatea,delta_effectivebf,delta_accumulatbv,delta_startsodium,delta_ufprofile,delta_sodiumpfile,delta_hpdelrate,delta_oCare_HR_mean,delta_oCare_HR_cv,delta_oCare_SpO2_mean,delta_oCare_SpO2_cv
0,3376.0,0.0,0.0,193.2,-0.4,49.80,249.0,35.5,35.60,13.700,0.4,1.3,0.04,200.80,500.0,521.0,50.8,49.0,150.0,1380.0,0.0,248.40,20.20,0.0,0.0,0.0,10.0,5.0,1090821.0,74159.0,171.0,89.0,55.0,74.0,1.0,0.0,209.0,57.386002,0.04653,71.644835,0.018368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3376.0,0.0,1.0,168.0,-0.4,48.40,249.0,35.5,35.66,13.720,0.4,1.3,0.10,190.00,500.0,522.6,51.0,48.2,750.0,1380.0,0.0,249.00,46.40,0.0,0.0,0.0,10.0,5.0,1090821.0,75159.0,171.0,89.0,55.0,74.0,1.0,0.0,209.0,57.386002,0.04653,71.644835,0.018368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3376.0,0.0,2.0,161.2,-0.4,52.40,249.0,35.5,35.70,13.720,0.4,1.3,0.20,180.00,500.0,524.2,51.0,48.4,600.0,1380.0,0.0,248.80,71.40,0.0,0.0,0.0,10.0,5.0,1090821.0,80158.0,171.0,89.0,55.0,74.0,1.0,0.0,209.0,57.386002,0.04653,71.644835,0.018368,0.0,0.0,0.0,0.0,0.0,-25.2,0.0,-1.4,0.0,0.0,0.06,0.02,0.0,0.0,0.06,-10.8,0.0,1.6,0.2,-0.8,600.0,0.0,0.0,0.6,26.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3376.0,0.0,3.0,169.8,-0.4,50.40,248.4,35.5,35.70,13.760,0.4,1.3,0.24,170.00,500.0,529.0,51.0,48.6,750.0,1380.0,0.0,248.40,96.80,0.0,0.0,0.0,10.0,5.0,1090821.0,81158.0,171.0,89.0,55.0,74.0,1.0,0.0,209.0,57.386002,0.04653,71.644835,0.018368,0.0,0.0,0.0,0.0,0.0,-6.8,0.0,4.0,0.0,0.0,0.04,0.00,0.0,0.0,0.10,-10.0,0.0,1.6,0.0,0.2,-150.0,0.0,0.0,-0.2,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3376.0,0.0,4.0,169.5,-0.5,48.25,249.0,35.5,35.70,13.725,0.4,1.3,0.30,160.75,500.0,525.0,51.0,48.5,750.0,1380.0,0.0,248.75,120.25,0.0,0.0,0.0,10.0,5.0,1090821.0,82158.0,171.0,89.0,55.0,74.0,1.0,0.0,209.0,57.386002,0.04653,71.644835,0.018368,0.0,0.0,0.0,0.0,0.0,8.6,0.0,-2.0,-0.6,0.0,0.00,0.04,0.0,0.0,0.04,-10.0,0.0,4.8,0.0,0.2,150.0,0.0,0.0,-0.4,25.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# result_valid.to_csv('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient-aggregate-features--format-IV.3-valid.csv', index=False)
result_valid.to_csv('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient-aggregate-features--format-IV.3--delta-all.csv', index=False)


In [ ]:
## test 

In [3]:
import csv
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

hmpcrt_no = [3376,7146,10380,15521,16827,18651,23278,32813,44488,45934,
            56075,57830,63579,90496,96961,119298,139980,142533,184883,216995,221211,221472,258260] #31384
                
#Patient_data_file_pretranslate_path
path = ('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/Patient_oCare_data_file/')

#bp_data_path
bp_data_path = ('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient_all_data_set/bp_data_1024.csv')

#machine_data_path
machine_data_path = ('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient_all_data_set/machine_data_1024_aggregate_by10min.csv')

#output_path
patient_aggregate_format_path = ('AN-NAN-Hypotension-datasets/Prediction_Hypo_IRB_Data_set/patient-aggregate-features--format-IV.2.csv')

#bp_data
with open(bp_data_path, encoding='utf-8') as f:
    inp = f.readlines()

bp_data = pd.DataFrame([row.strip().split(',') for row in inp[1:]], columns=inp[0].strip().split(','))
print(f'bp_data: {bp_data.shape}')

for c in bp_data.columns:
    bp_data[c] = bp_data[c].apply(lambda x: x.encode('utf-8').decode('utf-8-sig'))
bp_data.columns = [c.encode('utf-8').decode('utf-8-sig') for c in bp_data.columns]
bp_data['diastolic'] = np.where(bp_data['diastolic']=='', '68', bp_data['diastolic'])
for c in bp_data.columns:
    if c not in ['hmpcrtno','btrim','watch']:
        bp_data[c] = bp_data[c].astype(int)

#machine_data
machine_data = pd.read_csv(machine_data_path)
print(f'machine_data: {machine_data.shape}')
# for c in machine_data.columns:
#     if c not in ['hmpcrtno','btrim','ufprofile','sodiumpfile','watch']:
#         machine_data[c] = machine_data[c].astype(float)


bp_data: (5066, 11)
machine_data: (13179, 30)


In [10]:
# bp_data_hmpcrt_no.head()
bp_data_hmpcrt_no_daidate.head()
bp_data_hmpcrt_no_daidate_daitimes

# machine_data_hmpcrt_no.head()
# machine_data_hmpcrt_no_daidate.head()
machine_data_hmpcrt_no_daidate_daitimes[-5:]

oCare_data_hmpcrt_no_daidate.head()
point=23
machine_data_hmpcrt_no_daidate_daitimes[point]
[x[1] for x in enumerate(bp_data_hmpcrt_no_daidate_daitimes)]
if (machine_data_hmpcrt_no_daidate_daitimes[point] > min(bp_data_hmpcrt_no_daidate_daitimes)) & (machine_data_hmpcrt_no_daidate_daitimes[point] < max(bp_data_hmpcrt_no_daidate_daitimes)):
    # index 不該 -1 因為要對照下一個時間點
    index = next(x[0] for x in enumerate(bp_data_hmpcrt_no_daidate_daitimes) if x[1] > machine_data_hmpcrt_no_daidate_daitimes[point])

elif machine_data_hmpcrt_no_daidate_daitimes[point] <= min(bp_data_hmpcrt_no_daidate_daitimes):
    index = -1

elif machine_data_hmpcrt_no_daidate_daitimes[point] >= max(bp_data_hmpcrt_no_daidate_daitimes):
    index = len(bp_data_hmpcrt_no_daidate_daitimes)

else:
    print(hmpcrtno, daidate, machine_data_hmpcrt_no_daidate_daitimes[point], bp_data_hmpcrt_no_daidate_daitimes)

index

if point == 0:
    period = oCare_data_hmpcrt_no_daidate[oCare_data_hmpcrt_no_daidate['daitime'] < machine_data_hmpcrt_no_daidate_daitimes[point]]
    print(period)
else:
    period = oCare_data_hmpcrt_no_daidate[(oCare_data_hmpcrt_no_daidate['daitime'] > machine_data_hmpcrt_no_daidate_daitimes[point - 1] - 1) & (oCare_data_hmpcrt_no_daidate['daitime'] < machine_data_hmpcrt_no_daidate_daitimes[point])]
    print(period)
    

,hmpcrtno,btrim,daidate,daitime,systole,map,diastolic,pulserate,watch,Hypo_delta_20,Hypo_90mmHg
292,10380,A07,1090822,125608,139,92,64,83,4,0,0
293,10380,A07,1090822,132609,148,101,79,77,4,0,0
294,10380,A07,1090822,135609,141,86,66,76,4,0,0
295,10380,A07,1090822,142609,147,110,82,79,4,0,0
296,10380,A07,1090822,145609,155,107,73,80,4,0,0


[125608, 132609, 135609, 142609, 145609, 152610, 155610, 162610, 163810]

[160210.0, 161210.0, 162210.0, 163210.0, 164210.0]

,hmpcrtno,daidate,daitime,SpO2,HR
0,10380,1090822,125726,100,88
1,10380,1090822,125727,100,88
2,10380,1090822,125728,95,88
3,10380,1090822,125729,95,85
4,10380,1090822,125730,95,86


163210.0

[125608, 132609, 135609, 142609, 145609, 152610, 155610, 162610, 163810]

8

       hmpcrtno  daidate  daitime  SpO2  HR
12056     10380  1090822   162210    97  90
12057     10380  1090822   162211    97  90
12058     10380  1090822   162212    96  90
12059     10380  1090822   162213    96  91
12060     10380  1090822   162214    95  91
...         ...      ...      ...   ...  ..
12651     10380  1090822   163205    97  93
12652     10380  1090822   163206    97  93
12653     10380  1090822   163207    97  92
12654     10380  1090822   163208    97  92
12655     10380  1090822   163209    97  91

[600 rows x 5 columns]


In [5]:
# main
collect = dict()

for hmpcrtno in tqdm(hmpcrt_no[2:3]):

    collect[str(hmpcrtno)] = list()
    #存放該patient有oCare_data檔案紀錄的日期
    oCare_filedate_list = list()
    
    
    bp_data_hmpcrt_no = bp_data[bp_data['hmpcrtno']==str(hmpcrtno)]
    machine_data_hmpcrt_no = machine_data[machine_data['hmpcrtno']==hmpcrtno]
    oCare_data_hmpcrt_no = Path(path + str(hmpcrtno))
    oCare_data_hmpcrt_no_filelist = list(oCare_data_hmpcrt_no.glob("**/*.csv"))
    for File in oCare_data_hmpcrt_no_filelist: 
        file_date = (File.name)[13:20]
        oCare_filedate_list.append(file_date)
    print(oCare_filedate_list)
    
    for daidate in machine_data_hmpcrt_no.daidate.unique():
        
        bp_data_hmpcrt_no_daidate = bp_data_hmpcrt_no.query(" daidate == @daidate ")
        bp_data_hmpcrt_no_daidate_daitimes = bp_data_hmpcrt_no_daidate.daitime.values.tolist()
        machine_data_hmpcrt_no_daidate = machine_data_hmpcrt_no.query(" daidate == @daidate ")
        machine_data_hmpcrt_no_daidate_daitimes = machine_data_hmpcrt_no_daidate.daitime.values.tolist()
        
        if str(int(daidate)) in oCare_filedate_list:
            print(oCare_filedate_list.index(str(int(daidate))))
            oCare_data_hmpcrt_no_daidate = pd.read_csv(oCare_data_hmpcrt_no_filelist[oCare_filedate_list.index(str(int(daidate)))])
            for c in oCare_data_hmpcrt_no_daidate.columns:
                oCare_data_hmpcrt_no_daidate[c] = oCare_data_hmpcrt_no_daidate[c].astype(float)
                oCare_data_hmpcrt_no_daidate[c] = oCare_data_hmpcrt_no_daidate[c].astype(int)
            print(f'shape of oCare_data_hmpcrt_no_daidate: {oCare_data_hmpcrt_no_daidate.shape}')
#             oCare_data_hmpcrt_no_daidate_daitimes = oCare_data_hmpcrt_no_daidate.daitime.values.tolist()
        else:
            oCare_data_hmpcrt_no_daidate = pd.DataFrame(np.array([[0,0,0,0,0]]), columns=['hmpcrtno', 'daidate', 'daitime', 'SpO2', 'HR'])
        
        for point in range(len(machine_data_hmpcrt_no_daidate_daitimes)):
            
            if (machine_data_hmpcrt_no_daidate_daitimes[point] > min(bp_data_hmpcrt_no_daidate_daitimes)) & (machine_data_hmpcrt_no_daidate_daitimes[point] < max(bp_data_hmpcrt_no_daidate_daitimes)):
                # index 不該 -1 因為要對照下一個時間點
                index = next(x[0] for x in enumerate(bp_data_hmpcrt_no_daidate_daitimes) if x[1] > machine_data_hmpcrt_no_daidate_daitimes[point])
                
            elif machine_data_hmpcrt_no_daidate_daitimes[point] <= min(bp_data_hmpcrt_no_daidate_daitimes):
                # 對到第一個時間點的測量值
                index = 0
                
            elif machine_data_hmpcrt_no_daidate_daitimes[point] >= max(bp_data_hmpcrt_no_daidate_daitimes):
                # 接下來的測量值未知
                index = -1
                
            else:
                print(hmpcrtno, daidate, machine_data_hmpcrt_no_daidate_daitimes[point], bp_data_hmpcrt_no_daidate_daitimes)

                
            if point == 0:
                period = oCare_data_hmpcrt_no_daidate[oCare_data_hmpcrt_no_daidate['daitime'] < machine_data_hmpcrt_no_daidate_daitimes[point]]
                print(period)
            else:
                period = oCare_data_hmpcrt_no_daidate[(oCare_data_hmpcrt_no_daidate['daitime'] > machine_data_hmpcrt_no_daidate_daitimes[point - 1] - 1) & (oCare_data_hmpcrt_no_daidate['daitime'] < machine_data_hmpcrt_no_daidate_daitimes[point])]
                print(period)
                

            HR_mean = np.mean(period.HR)
            HR_cv = (np.std(period.HR) / HR_mean)
            SpO2_mean = np.mean(period.SpO2)
            SpO2_cv = (np.std(period.SpO2) / SpO2_mean)
            print(HR_mean, HR_cv, SpO2_mean, SpO2_cv)

            
            #append
            ##machine_data
            to_append = machine_data_hmpcrt_no_daidate[machine_data_hmpcrt_no_daidate['daitime']==machine_data_hmpcrt_no_daidate_daitimes[point]].iloc[0, :].values.tolist()
            ##bp_data
            if index == -1:
                for v in bp_data_hmpcrt_no_daidate.iloc[0, 4:].values:
                    to_append.append(np.nan)
            else:
                for v in bp_data_hmpcrt_no_daidate.iloc[index, 4:].values:
                    to_append.append(v)
            ##oCare_data
            for v in [HR_mean, HR_cv, SpO2_mean, SpO2_cv]:
                to_append.append(v)

                
            collect[str(hmpcrtno)].append(to_append)


            
#output
    result = pd.DataFrame(collect[str(3376)], columns=machine_data_hmpcrt_no_daidate.columns.tolist()+bp_data_hmpcrt_no_daidate.columns.tolist()[4:]+['oCare_HR_mean', 'oCare_HR_cv', 'oCare_SpO2_mean', 'oCare_SpO2_cv'])
    print(result.head())



  0%|                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

['1090822', '1090825', '1090917', '1090919', '1090922', '1090924', '1090929', '1091008', '1091010', '1091013', '1091015']
0
shape of oCare_data_hmpcrt_no_daidate: (14479, 5)
Empty DataFrame
Columns: [hmpcrtno, daidate, daitime, SpO2, HR]
Index: []
nan nan nan nan
Empty DataFrame
Columns: [hmpcrtno, daidate, daitime, SpO2, HR]
Index: []
nan nan nan nan
     hmpcrtno  daidate  daitime  SpO2  HR
0       10380  1090822   125726   100  88
1       10380  1090822   125727   100  88
2       10380  1090822   125728    95  88
3       10380  1090822   125729    95  85
4       10380  1090822   125730    95  86
..        ...      ...      ...   ...  ..
277     10380  1090822   130203    98  81
278     10380  1090822   130204    98  81
279     10380  1090822   130205    98  81
280     10380  1090822   130206    98  80
281     10380  1090822   130207    98  80

[282 rows x 5 columns]
82.16312056737588 0.032690908419248134 95.3936170212766 0.013780282720770882
     hmpcrtno  daidate  daitime  SpO2  HR

IndexError: single positional indexer is out-of-bounds

In [6]:
for hmpcrtno in hmpcrt_no[1:]:
    result = pd.concat([result, pd.DataFrame(collect[str(hmpcrtno)], columns=machine_data_hmpcrt_no_daidate.columns.tolist()+bp_data_hmpcrt_no_daidate.columns.tolist()[4:]+['oCare_HR_mean', 'oCare_HR_cv', 'oCare_SpO2_mean', 'oCare_SpO2_cv'])])


print(f'result: {result.shape}')
print(result.head())
print(result.isnull().sum())
# result.to_csv(patient_aggregate_format_path, index=False)


NameError: name 'result' is not defined